In [ ]:
# Libraries & Settings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2 
import sys
import requests as r
import os

here_maps_api_key = '###'

In [ ]:
translation_table = str.maketrans("çığşüöÇİĞŞÜÖ", "cigsuoCIGSUO")

### Toplanma Alanları

In [ ]:
toplanma = pd.read_csv('Datasets/Toplanma_Alanları/ToplanmaAlanlari.csv',
                       header=0,
                       names=['id', 'kapasite', 'ilce_uavt',
                              'fonksiyon', 'shape_len', 'shape_area',
                              'lat', 'lon', 'ilce'])

toplanma['ilce'] = toplanma['ilce'].apply(lambda x: x.translate(translation_table).lower())

### Mahalle Merkezleri Yoğunlukları

In [ ]:
density = pd.read_csv('Datasets/Periodic_Yoğunluk/population_density_time_period.csv',
                      header=0,
                      names=['date', 'time_interval', 'mahalle', 'ilce', 'nufus'])

density.dropna(inplace=True)

In [ ]:
density['mahalle'] = density['mahalle'].apply(lambda x: x.translate(translation_table).lower())
density['ilce'] = density['ilce'].apply(lambda x: x.translate(translation_table).lower())
density['ilce'] = density['ilce'].apply(lambda x: x.replace('eyup', 'eyupsultan'))
density = density[density['ilce'].isin(toplanma['ilce'].to_list())]

### Mahalle Merkezleri Koordinatları

In [ ]:
def get_coordinates(adres):
    import requests as r
    
    base_url = 'https://geocode.search.hereapi.com/v1/geocode'
    
    params = {
        'apiKey': here_maps_api_key,
        'q': adres
    }
    
    response = r.session().get(base_url, params=params).json()
    
    lat = response['items'][0]['position']['lat']
    lon = response['items'][0]['position']['lng']
    
    return lat, lon
    

In [ ]:
density['adres'] = density['mahalle'] + str('+') + density['ilce']

In [ ]:
unique_adres = pd.DataFrame(density['adres'].unique(), columns=['adres'])

In [ ]:
unique_adres_list_coords = []
for index, row in unique_adres[415:].iterrows():
    print(index, 'done')
    try:
        lat, lon = get_coordinates(row.adres)
        unique_adres_list_coords.append([row.adres, lat, lon])
    except Exception as e:
        print(e)
        continue

In [ ]:
unique_adres_lat_lon = pd.DataFrame(unique_adres_list_coords,columns=['adres', 'lat', 'lon'])

In [ ]:
unique_adres_lat_lon_2 = pd.DataFrame(unique_adres_list_coords,columns=['adres', 'lat', 'lon'])

In [ ]:
add = pd.DataFrame([['inonu+bagcilar', 41.030168, 28.851756], 
                    ['orta+pendik', 40.890313, 29.242589]], columns=['adres', 'lat', 'lon'])

In [ ]:
unique_adres_lat_lon.loc[unique_adres_lat_lon['adres'] == 'merkez+beylikduzu', 'lat'] = 40.986465
unique_adres_lat_lon.loc[unique_adres_lat_lon['adres'] == 'merkez+beylikduzu', 'lon'] = 28.643884

In [ ]:
unique_adres_lat_lon.loc[unique_adres_lat_lon['adres'] == 'kazim karabekir+sultangazi', 'lat'] = 41.100734
unique_adres_lat_lon.loc[unique_adres_lat_lon['adres'] == 'kazim karabekir+sultangazi', 'lon'] = 28.912222

In [ ]:
unique_adres_lat_lon_2.loc[unique_adres_lat_lon_2['adres'] == 'ataturk+esenyurt', 'lat'] = 41.028068
unique_adres_lat_lon_2.loc[unique_adres_lat_lon_2['adres'] == 'ataturk+esenyurt', 'lon'] = 28.639226

In [ ]:
unique_adres_lat_lon_2.loc[unique_adres_lat_lon_2['adres'] == 'inonu+sisli', 'lat'] = 41.043558
unique_adres_lat_lon_2.loc[unique_adres_lat_lon_2['adres'] == 'inonu+sisli', 'lon'] = 28.984346

In [ ]:
unique_adres_lat_lon_2.loc[unique_adres_lat_lon_2['adres'] == 'rustempasa+fatih', 'lat'] = 41.017253 
unique_adres_lat_lon_2.loc[unique_adres_lat_lon_2['adres'] == 'rustempasa+fatih', 'lon'] = 28.970757

In [ ]:
final_df = unique_adres_lat_lon.append(unique_adres_lat_lon_2, ignore_index=True).append(add, ignore_index=True)

In [ ]:
final_df.to_csv('mahalle_merkezleri_coords.csv', index=False, header=True)